In [1]:
### Set colaboratory True to run in Google Colaboratory. 
try:
    ## Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    ## Specify a directory in Google Drive
    dir = '/content/drive/My Drive/Colab Notebooks/LGBM_Optuna'
    os.chdir(dir)
    
    ## Check the environment info
    print('## Current working directory: ', os.getcwd())
    #os.listdir()
    print('## Check the uptime. (Google Colab reboots every 12 hours)')
    !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
    print('## Check the GPU info')
    !nvidia-smi
    print('## Check the Python version') 
    !python --version
    print('## Check the OS') 
    !cat /etc/issue
    print('## Check the memory')
    !free -h
    print('## Check the disk')
    !df -h
except:
    print('Run the code assuming the environment is not Google Colab.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
## Current working directory:  /content/drive/My Drive/Colab Notebooks/LGBM_Optuna
## Check the uptime. (Google Colab reboots every 12 hours)
Uptime is 2.0107 hours (7238.53 sec)
## Check the GPU info
Mon Feb  4 09:28:55 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    71W / 149W |    462MiB / 11441MiB |      0%      Default 

In [2]:
!pip install optuna
import optuna

Using TensorFlow backend.


https://stackoverflow.com/questions/49606886/installing-gpu-support-for-lightgbm-on-google-collab

In [3]:
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd LightGBM
!mkdir build
!cd build
!cmake ./LightGBM
!make -j4

fatal: destination path 'LightGBM' already exists and is not an empty directory.
/content/drive/My Drive/Colab Notebooks/LGBM_Optuna/LightGBM
mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.3.0
-- The CXX compiler identification is GNU 7.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Configuring done
-- Generating done
-- Build files have bee

In [4]:
!git clone --recursive https://github.com/Microsoft/LightGBM.git
%cd LightGBM/python-package
!python3 setup.py install --gpu

fatal: destination path 'LightGBM' already exists and is not an empty directory.
/content/drive/My Drive/Colab Notebooks/LGBM_Optuna/LightGBM/LightGBM/python-package
running install
INFO:LightGBM:Starting to compile the library.
INFO:LightGBM:Starting to compile with CMake.
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
running egg_info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
running install_lib
INFO:LightGBM:Installing lib_lightgbm from: ['../lib_lightgbm.so', 'compile/lib_lightgbm.so']
copying ../lib_lightgbm.so -

In [0]:

import lightgbm as lgb
import optuna, os, uuid, pickle
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [0]:

data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data["data"], data["target"], test_size=0.3, random_state=19)

In [0]:


def objectives(trial):
    trial_uuid = str(uuid.uuid4())
    trial.set_user_attr("uuid", trial_uuid)

    params = {
        'boosting_type': trial.suggest_categorical('boosting', ['gbdt', 'dart', 'goss']),
        'objective': 'binary',
        'metric': {'binary', 'binary_error', 'auc'},
        'num_leaves': trial.suggest_int("num_leaves", 10, 500),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-5, 1),
        'feature_fraction': trial.suggest_uniform("feature_fraction", 0.0, 1.0),
        'device' : 'gpu',
        'verbose' : 0
    }
    if params['boosting_type'] == 'dart':
        params['drop_rate'] = trial.suggest_loguniform('drop_rate', 1e-8, 1.0)
        params['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)
    if params['boosting_type'] == 'goss':
        params['top_rate'] = trial.suggest_uniform('top_rate', 0.0, 1.0)
        params['other_rate'] = trial.suggest_uniform('other_rate', 0.0, 1.0 - params['top_rate'])

    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "binary_logloss") # 正式名で呼ばないとダメなので注意
    gbm = lgb.train(params, lgb.Dataset(X_train, y_train), num_boost_round=500,
                    valid_sets=lgb.Dataset(X_test, y_test), callbacks=[pruning_callback])

    y_pred_train = np.rint(gbm.predict(X_train))
    y_pred_test = np.rint(gbm.predict(X_test))
    error_train = 1.0 - accuracy_score(y_train, y_pred_train)
    error_test = 1.0 - accuracy_score(y_test, y_pred_test)

    trial.set_user_attr("train_error", error_train)
    trial.set_user_attr("test_error", error_test)

    if not os.path.exists("lgb_output"):
        os.mkdir("lgb_output")
    with open("lgb_output/"+f"{trial_uuid}.pkl", "wb") as fp:
        pickle.dump(gbm, fp)

    return error_test



In [8]:
#study = optuna.create_study()
study = optuna.create_study(storage="sqlite:///brestcancer_lgb.db", study_name="brestcancer_lgb", load_if_exists=True)
study.optimize(objectives, n_trials=100)


[I 2019-02-04 09:34:57,258] Using an existing study with name 'brestcancer_lgb' instead of creating a new one.


[1]	valid_0's binary_logloss: 0.682525	valid_0's binary_error: 0.409357	valid_0's auc: 0.778925
[2]	valid_0's binary_logloss: 0.682512	valid_0's binary_error: 0.409357	valid_0's auc: 0.966549
[3]	valid_0's binary_logloss: 0.682502	valid_0's binary_error: 0.409357	valid_0's auc: 0.963791
[4]	valid_0's binary_logloss: 0.682492	valid_0's binary_error: 0.409357	valid_0's auc: 0.95983
[5]	valid_0's binary_logloss: 0.682482	valid_0's binary_error: 0.409357	valid_0's auc: 0.961952
[6]	valid_0's binary_logloss: 0.682479	valid_0's binary_error: 0.409357	valid_0's auc: 0.970297
[7]	valid_0's binary_logloss: 0.682468	valid_0's binary_error: 0.409357	valid_0's auc: 0.979632
[8]	valid_0's binary_logloss: 0.682465	valid_0's binary_error: 0.409357	valid_0's auc: 0.980481
[9]	valid_0's binary_logloss: 0.682461	valid_0's binary_error: 0.409357	valid_0's auc: 0.985997
[10]	valid_0's binary_logloss: 0.682458	valid_0's binary_error: 0.409357	valid_0's auc: 0.987412
[11]	valid_0's binary_logloss: 0.682447	

[I 2019-02-04 09:35:55,849] Finished a trial resulted in value: 0.4093567251461988. Current best value is 0.4093567251461988 with parameters: {'boosting': 'goss', 'feature_fraction': 0.05229727775244175, 'learning_rate': 1.5540589129357155e-05, 'num_leaves': 140, 'other_rate': 0.16035223666727205, 'top_rate': 0.006975942278953817}.


[1]	valid_0's binary_logloss: 0.682305	valid_0's binary_error: 0.409357	valid_0's auc: 0.958204
[2]	valid_0's binary_logloss: 0.682077	valid_0's binary_error: 0.409357	valid_0's auc: 0.958982
[3]	valid_0's binary_logloss: 0.681859	valid_0's binary_error: 0.409357	valid_0's auc: 0.968741
[4]	valid_0's binary_logloss: 0.681637	valid_0's binary_error: 0.409357	valid_0's auc: 0.96959
[5]	valid_0's binary_logloss: 0.681405	valid_0's binary_error: 0.409357	valid_0's auc: 0.97058
[6]	valid_0's binary_logloss: 0.681188	valid_0's binary_error: 0.409357	valid_0's auc: 0.972984
[7]	valid_0's binary_logloss: 0.680962	valid_0's binary_error: 0.409357	valid_0's auc: 0.974116
[8]	valid_0's binary_logloss: 0.680738	valid_0's binary_error: 0.409357	valid_0's auc: 0.974823
[9]	valid_0's binary_logloss: 0.680515	valid_0's binary_error: 0.409357	valid_0's auc: 0.974399
[10]	valid_0's binary_logloss: 0.680292	valid_0's binary_error: 0.409357	valid_0's auc: 0.975248
[11]	valid_0's binary_logloss: 0.680061	v

[I 2019-02-04 09:37:02,681] Finished a trial resulted in value: 0.4093567251461988. Current best value is 0.4093567251461988 with parameters: {'boosting': 'goss', 'feature_fraction': 0.05229727775244175, 'learning_rate': 1.5540589129357155e-05, 'num_leaves': 140, 'other_rate': 0.16035223666727205, 'top_rate': 0.006975942278953817}.


...

[I 2019-02-04 10:20:41,790] Setting trial status as TrialState.PRUNED. Trial was pruned at iteration 1.


[1]	valid_0's binary_logloss: 0.680936	valid_0's binary_error: 0.409357	valid_0's auc: 0.958204


[I 2019-02-04 10:20:45,785] Setting trial status as TrialState.PRUNED. Trial was pruned at iteration 1.


[1]	valid_0's binary_logloss: 0.680495	valid_0's binary_error: 0.409357	valid_0's auc: 0.958204


[I 2019-02-04 10:20:50,127] Setting trial status as TrialState.PRUNED. Trial was pruned at iteration 1.


In [9]:

print(study.best_params)
print(study.best_value)

print(study.best_trial.user_attrs)

df = study.trials_dataframe()
df.to_csv("optuna_lgb.csv")

{'boosting': 'dart', 'drop_rate': 0.000258094911594383, 'feature_fraction': 0.5643043538244379, 'learning_rate': 0.042812716624904296, 'num_leaves': 419, 'skip_drop': 0.5749417546199863}
0.04678362573099415
{'test_error': 0.04678362573099415, 'train_error': 0.0, 'uuid': '43f49e7c-02db-4524-beca-d6fee16677c8'}
